## INFSYS 722 - Iteration 4
ID: 168395378
<br/>
Author: Paolo Gabriel Averia Delos Reyes

### 01 - Business Understanding

This section, we describe the business goals and objectives.

Our goal is o study and analyze global water treatment and water consumption. From there, compare the Philippines versus top countries and identify where the Philippines can improve on.

### 02 - Data Understanding

Loading the dataset to grasp what the data looks like.

#### Library imports

In [2]:
# PySpark libraries
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pdel145_iter4').getOrCreate()

# Visualisation
import matplotlib.pyplot as plt

#### Loading the datasets

In [5]:
# Freshwater dataset
freshwater_df = spark.read.csv('data/freshwater.csv', inferSchema=True, header=True)

In [6]:
# Checking for its datatypes
freshwater_df.printSchema()

root
 |-- SDG: integer (nullable = true)
 |-- SDG target: double (nullable = true)
 |-- SDG indicator: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- Indicator name: string (nullable = true)
 |-- Geographical area code: string (nullable = true)
 |-- Geographical area name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Value: double (nullable = true)
 |-- Time detail: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Footnote: string (nullable = true)
 |-- Type of data: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Age group: string (nullable = true)
 |-- Bounds: string (nullable = true)
 |-- Frequency: string (nullable = true)
 |-- Level/Status: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Type of reporting: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SDG 6 Data portal level: string (nullable = true)



In [9]:
# Viewing what the dataset contains
freshwater_df.show()

+---+----------+-------------+--------------+--------------------+----------------------+----------------------+----+-----+-----------+--------------------+--------+--------------------+----------+---------+------+---------+------------+--------------------+-----------------+----+-----------------------+
|SDG|SDG target|SDG indicator|Indicator Code|      Indicator name|Geographical area code|Geographical area name|Year|Value|Time detail|              Source|Footnote|        Type of data|     Units|Age group|Bounds|Frequency|Level/Status|            Location|Type of reporting| Sex|SDG 6 Data portal level|
+---+----------+-------------+--------------+--------------------+----------------------+----------------------+----+-----+-----------+--------------------+--------+--------------------+----------+---------+------+---------+------------+--------------------+-----------------+----+-----------------------+
|  6|       6.4|        6.4.2| ER_H2O_STRESS|Level of water st...|               C

In [5]:
# Same goes for the rest of the datasets that are of file format .csv
impr_sanit_df = spark.read.csv('data/improved_sanitation.csv', inferSchema=True, header=True)
renewable_df = spark.read.csv('data/renewable.csv', inferSchema=True, header=True)
tot_popul_df = spark.read.csv('data/total_population.csv', inferSchema=True, header=True)
safe_manage_drink_df = spark.read.csv('data/safely_managed_drinking.csv', inferSchema=True, header=True)
safe_manage_sanit_df = spark.read.csv('data/safely_managed_sanitation.csv', inferSchema=True, header=True)